## Real Time Testing

In [4]:
# ! py -3.9 -m pip install cvzone==1.5.6
# ! py -3.9 -m pip install opencv-python==4.7.0.72
# ! py -3.9 -m pip install numpy==1.26.2
# ! py -3.9 -m pip install mediapipe==0.9.1.0
# ! py -3.9 -m pip install tensorflow==2.11.0
# ! py -3.9 -m pip install wordninja==2.0.0
# ! py -3.9 -m pip install transformers==4.35.2
# ! py -3.9 -m pip install kivy==2.1.0
# ! py -3.9 -m pip install kivymd==1.1.1
! py -3.9 -m pip install gtts
! py -3.9 -m pip install deep-translator
! py -3.9 -m pip install pyttsx3
! py -3.9 -m pip install playsound
# ! py -3.9 -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# ! py -3.9 -m pip list

  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     -------------------------------------- 143.0/143.0 kB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
   ---------------------------------------- 42.3/42.3 kB ? eta 0:00:00


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


   ---------------------------------------- 0.0/184.3 kB ? eta -:--:--
   --------------------------------------- 184.3/184.3 kB 11.6 MB/s eta 0:00:00


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7049 sha256=9835754140743d405f318f450ad4fa608b0873f0893e4377e1298167af3dd347
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ba\39\54\c8f7ff9a88a644d3c58b4dec802d90b79a2e0fb2a6b884bf82
Successfully built playsound


In [1]:
from collections import deque
from concurrent.futures import ThreadPoolExecutor
from time import time
from cvzone import FPS

import cv2
import numpy as np
from itertools import chain
from time import time

from cvzone.HandTrackingModule import HandDetector
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.PoseModule import PoseDetector

ModuleNotFoundError: No module named 'cvzone'

In [15]:
## 2. Feature Extraction (Hand+Face+Pose Detection)
# Flatten a 2d np array into 1d array
def flatten2dList(arr, dataType=int):
    return np.fromiter(chain.from_iterable(arr), dataType)

# Get the largest absolute value in an np array
def getAbsLargestVal(arr):
    return np.max(np.abs(arr))

# Offset and normalize the landmark list
# Returns a 1d numpy array
def preprocess_landmarks(landmark_list):    
    landmark_list = np.array(landmark_list, dtype=float)
    origin = landmark_list[0]
    
    # Offset every point with respect to the first point
    # Convert to 1D-array
    new_landmark_list = (landmark_list - origin).ravel()
    
    # Get highest absolute value
    largest_value = getAbsLargestVal(new_landmark_list)
    
    # Normalization
    if largest_value != 0:
        return new_landmark_list / largest_value
    return new_landmark_list

# Offset and normalize a BBOX list (BBOX = Bounding Box, used in face and hand detection)
# Returns a 1d numpy array
def preprocess_bbox(bbox, frameSize):
    bbox = np.array(bbox, dtype=float)
    # Convert 3rd and 4th element into coordinates instead of width/height
    bbox[2] = bbox[0] + bbox[2]
    bbox[3] = bbox[1] + bbox[3]

    # Normalize against frame size
    bbox[0] /= frameSize[0]
    bbox[1] /= frameSize[1]
    bbox[2] /= frameSize[0]
    bbox[3] /= frameSize[1]

    return bbox

# Normalize a center vertex (a list of 2 elements)
# Returns a 1d numpy array
def preprocess_center(center, frameSize):
    center = np.array(center, dtype=float)
    center[0] /= frameSize[0]
    center[1] /= frameSize[1]
    return center

# Preprocess (Offset and normalize) the body's landmark list, bbox and center
def preprocess_body_part(bodyPart, frameSize):
    bodyPart['lmList'] = preprocess_landmarks(bodyPart['lmList'])
    bodyPart['bbox'] = preprocess_bbox(bodyPart['bbox'], frameSize)
    bodyPart['center'] = preprocess_center(bodyPart['center'], frameSize)
    return bodyPart

# Function to generate empty/placeholder data for a hand 
# Used when a hand is not detected in frame
def generate_empty_hand(type):
    return {
        'lmList': np.zeros(21 * 3, dtype=int), 
        'bbox': np.zeros(4, dtype=float), 
        'center': np.zeros(2, dtype=float), 
        'type': type
    }

# Select the best matching face, aka the one with the best score (clarity)
# and closest to the center of the screen
# Since the Neural network will be design to only accept one face
def select_best_matching_face(faces, frameSize):
    if not faces or len(faces) == 0:
        return False
    elif len(faces) == 1:
        return faces[0]
    
    def difference(a, b):
        return ((a[0] - b[0])**2) + ((a[1] - b[1])**2)
    
    frameCenter = (frameSize[0] / 2, frameSize[1] / 2)

    best_score = faces[0]
    best_center = faces[0]
    center_diff = difference(faces[0]['center'], frameCenter)

    for each in faces:
        if difference(each['center'], frameCenter) < center_diff:
            best_center = each
        if each['score'][0] > best_score['score'][0]:
            best_score = each
    
    if best_center['score'][0] > 0.5:
        return best_center
    return best_score

# Flatten everything
def flattenDetectionResult(obj):
    # return np.fromiter(chain.from_iterable([obj['lmList'], obj['bbox'], obj['center']]), float)
    return np.concatenate([obj['lmList'], obj['bbox'], obj['center']])

In [16]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def featureExtractionV3(handDetector, faceDetector, poseDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    # Pose Detection
    # **We only use the first 23 out of the total 33 landmark points 
    #   as those represent the lower half body and are irrelevant to sign language interpretation
    def detectPose(poseDetector, frame, draw):
        frame = poseDetector.findPose(frame, draw=draw)
        results, _ = poseDetector.findPosition(frame, bboxWithHands=False)
        if results:
            results = preprocess_landmarks(results[:23])
        else:
            results = np.zeros(23, dtype=int)
        return results
    
    # Face Detection
    def detectFace(faceDetector, frame, frameSize, draw):
        frame, results = faceDetector.findFaces(frame, draw=draw)
        if results:
            results = select_best_matching_face(results, frameSize)
            results['bbox'] = preprocess_bbox(results['bbox'], frameSize)
            results['center'] = preprocess_center(results['center'], frameSize)
        else:
            results = {
                'bbox': np.zeros(4, dtype=float), 
                'center': np.zeros(2, dtype=float)
            }
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        t2 = executor.submit(detectPose, poseDetector, frame, draw)
        t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            t2.result(), 
            t3.result()['bbox'],
            t3.result()['center'],
            t3.result()['center'] - t1.result()[0]['center'],
            t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [17]:
# Detects hands, face & pose, 
# convert them into normalized landmark/keypoint coordinates in a 1D-array, 
# and also returns the frame with the landmark connections drawn onto it

# Improved/Parallelised version
def staticFeatureExtraction(handDetector, frame, draw=True):
    def detectHands(handDetector, frame, frameSize, draw):
        results = None
        # Hand Detection
        if (draw):
            results, frame = handDetector.findHands(frame, draw=draw)
        else:
            results = handDetector.findHands(frame, draw=draw)

        if not results:
            results = [generate_empty_hand('Left'), generate_empty_hand('Right')]
        elif len(results) == 1:
            if (results[0]['type'] == 'Left'):
                results[0] = preprocess_body_part(results[0], frameSize)
                results.append(generate_empty_hand('Right'))
            else:
                results[0] = preprocess_body_part(results[0], frameSize)
                results.insert(0, generate_empty_hand('Left'))                         
        else:
            results[0] = preprocess_body_part(results[0], frameSize)
            results[1] = preprocess_body_part(results[1], frameSize)
        return results

    frameSize = (frame.shape[1], frame.shape[0])
    with ThreadPoolExecutor() as executor:
        t1 = executor.submit(detectHands, handDetector, frame, frameSize, draw)
        # t2 = executor.submit(detectPose, poseDetector, frame, draw)
        # t3 = executor.submit(detectFace, faceDetector, frame, frameSize, draw)
        
        # Convert results into 1D-array
        detectionResults = flatten2dList([
            flattenDetectionResult(t1.result()[0]), 
            flattenDetectionResult(t1.result()[1]), 
            # t2.result(), 
            # t3.result()['bbox'],
            # t3.result()['center'],
            # t3.result()['center'] - t1.result()[0]['center'],
            # t3.result()['center'] - t1.result()[1]['center']
        ], dataType=float)

        return detectionResults, frame

In [18]:
import keras
from files_io import readActionLabels

# Load the model from the H5 file
model = keras.models.load_model('../static_recognition/models/static_model_V1.h5')
# model = keras.models.load_model('../static_recognition/models/static_model.keras')

# Print the summary before loading
model.summary()

static_labels = readActionLabels()
static_labels

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 136, 32)           128       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 68, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            6208      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 33, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 31, 128)           24704     
                                                                 
 flatten (Flatten)           (None, 3968)              0

['A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10']

In [19]:
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# Detectors
handDetector = HandDetector(detectionCon=0.5, maxHands=2)
faceDetector = FaceDetector(minDetectionCon=0.5)
poseDetector = PoseDetector(detectionCon=0.5)
fpsReader = FPS()

timeStats = []

try:
    predictionHistory = deque()
    detectionThreshold = 0.999    
    predictionCooldown = 0.5
    appendCooldown = 1.0
    # the first time append should eliminate predictionCooldown
    lastAppendTime = time() + appendCooldown
    lastDetectTime = time() + predictionCooldown

    while True:
        startTime = time()

        # Read from camera
        success, frame = cam.read()
        if not success:
            raise Exception("No Frames Read")
        frame = cv2.flip(frame, 1)

        # Hand Detection
        inputValue, frame = staticFeatureExtraction(
            handDetector, frame)
        
        # detectionResults, frame = featureExtractionV3(
        #     handDetector, faceDetector, poseDetector, frame)
        
        detectionResults = np.expand_dims(
            inputValue, axis=0) # reshape detection result
        
        if time() <= lastDetectTime + predictionCooldown:
            pass
        else:
            predictionResults = model.predict(
                x=detectionResults,
                verbose=0,
                use_multiprocessing=True,
                workers=4
            )[0]
            lastDetectTime = time()
            
            print(predictionResults)
            
            # Get predicted character and accuracy
            predCharacter = static_labels[np.argmax(predictionResults)]
            predAccuracy = predictionResults[np.argmax(predictionResults)]
        
            cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)          
            
            if predAccuracy >= detectionThreshold:            
                # If predictionHistory is not empty
                # If predCharacter is the same as the last appended character
                # Check if 0.5 seconds have passed since the last append
                if predictionHistory and predCharacter == predictionHistory[-1] and time() <= lastAppendTime + appendCooldown:
                    # Do nothing, don't append
                    pass
                else:
                    predictionHistory.append(predCharacter)
                    # Reset the timestamp when a new character is detected
                    lastAppendTime = time()

                if len(predictionHistory) > 5:
                        predictionHistory.popleft()  
                        
        # cv2.putText(frame, (predCharacter + ":"+ str(predAccuracy)), (15, 70),
        #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
                        
        cv2.putText(frame, str(predictionHistory), (15, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
        

        fps, frame = fpsReader.update(frame, pos=(
            950, 80), color=(0, 255, 0),scale=5, thickness=5)

        # fps, frame = fpsReader.update(frame, pos=(
        #     950, 80), color=(0, 255, 0), scale=5, thickness=5)

        
        # Show resulting frame
        # cv2.putText(frame, f'Training #{training + 1} for \'{action}\'', (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
        cv2.imshow("Sign Language Recognition Prototype", frame)

        timeStats.append(time() - startTime)

        keyPressed = cv2.waitKey(1)
        # Stop Program when pressed 'Esc'
        if keyPressed == 27:
            raise Exception("Finished")
        elif keyPressed == ord('c'):
            predictionHistory.clear()
            lastAppendTime = time() + predictionCooldown


except Exception as e:
    cam.release()
    cv2.destroyAllWindows()
    print(e)

cam.release()
cv2.destroyAllWindows()

[1.37623454e-06 1.46660716e-06 7.81543552e-10 1.50382064e-07
 2.05155866e-05 6.09210656e-05 5.27407713e-02 1.31454738e-03
 5.88160183e-05 1.35719753e-03 2.46477128e-07 3.33097161e-10
 7.06416685e-08 1.14193945e-05 2.89141121e-06 1.40714483e-05
 6.37933317e-06 1.44625503e-08 4.33857320e-03 2.66098118e-06
 1.21854214e-07 2.55941031e-05 1.59500507e-06 4.00029775e-03
 4.30734133e-07 6.40908502e-06 3.12973210e-03 8.32477090e-05
 8.59810039e-04 7.94393713e-07 6.82757445e-07 2.31001934e-07
 1.55312417e-04 3.90924804e-08 5.44565466e-07 9.31803107e-01]
[1.4954883e-06 7.6643659e-07 3.8952899e-10 1.5261443e-07 4.7430576e-06
 1.1678073e-04 2.0310499e-02 4.7071616e-04 2.4955703e-05 5.7691912e-04
 1.8420405e-07 3.3297595e-10 5.2803035e-08 6.3493944e-06 2.1486924e-06
 9.8728124e-06 3.4562627e-06 7.1387163e-09 1.8926673e-03 1.4151939e-06
 7.0703976e-08 1.3100661e-05 8.2361237e-07 2.0876431e-03 5.5089919e-07
 1.7118278e-06 1.1796477e-03 7.7193901e-05 2.1985863e-04 5.6408328e-07
 6.9816332e-07 1.7877865

In [20]:
timeStats[10:]

[0.06218910217285156,
 0.10654020309448242,
 0.10498833656311035,
 0.08688926696777344,
 0.15842938423156738,
 0.05167722702026367,
 0.08908200263977051,
 0.08889389038085938,
 0.10922455787658691,
 0.09494209289550781,
 0.14324259757995605,
 0.0656285285949707,
 0.09074091911315918,
 0.09238553047180176,
 0.11070942878723145,
 0.09563779830932617,
 0.14750361442565918,
 0.0523836612701416,
 0.10111069679260254,
 0.09392738342285156,
 0.10452151298522949,
 0.10024595260620117,
 0.1509408950805664,
 0.05222582817077637,
 0.0936136245727539,
 0.09185624122619629,
 0.09932518005371094,
 0.08919477462768555,
 0.16465520858764648,
 0.04788398742675781,
 0.0834803581237793,
 0.11100268363952637,
 0.09181380271911621,
 0.09565258026123047,
 0.14389538764953613,
 0.05892181396484375,
 0.09246611595153809,
 0.09319519996643066,
 0.1189725399017334,
 0.08735036849975586,
 0.1493387222290039,
 0.0569615364074707,
 0.09237504005432129,
 0.0978083610534668,
 0.09218716621398926,
 0.1093502044677734